<a href="https://colab.research.google.com/github/joseffaghihi/PACE-PEACE-dicrete-and-Continious/blob/main/Linear_Peace_(Synthetic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
pip install vegas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sympy
from sklearn.metrics import mean_squared_error
import vegas

In [86]:
n = 1000000                                                      # The number of samples
mean_real_0, mean_real_1, mean_real_2 = 0, 0, 0                  # Specifying the means of data
mu_real_0, mu_real_1, mu_real_2 = 1.5, 1, 0.8                    # Specifying the variances of data
coeff_real = [0,-1,1.05,2]                                       # Specifying the coefficients of the polynomial of degeee 1 whose variables are x0, x1 and x2. The order of the terms in this polynomial is as follows: 1, x0, x1, x2.
np.random.seed(0)
x0 = np.random.normal(mean_real_0, mu_real_0, n).flatten()       # Generating x0 from a normal distribution with the above mean and variance. 
x1 = np.random.normal(mean_real_1, mu_real_1, n).flatten()       # Generating x1 from a normal distribution with the above mean and variance.
x2 = np.random.normal(mean_real_2, mu_real_2, n).flatten()       # Generating x2 from a normal distribution with the above mean and variance.

# Generating y from the polynomial with the above coefficients. Indeed, a random error is applied on the coefficients. 
y = (0.01*np.random.rand(1)[0]+coeff_real[0]) + (0.01*np.random.rand(1)[0]+coeff_real[1])*x0 + (0.01*np.random.rand(1)[0]+coeff_real[2])*x1 + (0.01*np.random.rand(1)[0]+coeff_real[3])*x2

# Defining a dataframe from the above data.
data = np.zeros((n,4))
data[:,0], data[:,1], data[:,2], data[:,3] = [x0,x1,x2, y]
df = pd.DataFrame(data)
cols = ["x0", "x1", "x2", "y"]
df.columns = cols
df.head()

,x0,x1,x2,y
0,2.646079,0.514247,-0.186862,-2.443247
1,0.600236,1.112865,0.772854,2.138009
2,1.468107,0.675024,0.355203,-0.020542
3,3.361340,-0.560557,1.336306,-1.232948
4,2.801337,-0.834898,0.557275,-2.529083


In [87]:
# Renaming the columns x0, x1, x2 to 0,1 and 2, respectively.

df = df.rename(columns={'x0':0, 'x1':1, 'x2':2})
df.head()

,0,1,2,y
0,2.646079,0.514247,-0.186862,-2.443247
1,0.600236,1.112865,0.772854,2.138009
2,1.468107,0.675024,0.355203,-0.020542
3,3.361340,-0.560557,1.336306,-1.232948
4,2.801337,-0.834898,0.557275,-2.529083


In [88]:
# Calculating the means and the standard deviations of each column of the data.

df_ar = df.to_numpy()
mean = np.mean(df_ar.transpose()[0]), np.mean(df_ar.transpose()[1]) , np.mean(df_ar.transpose()[2])
sd = np.sqrt(float(np.cov(df_ar.transpose()[0]))), np.sqrt(float(np.cov(df_ar.transpose()[1]))), np.sqrt(float(np.cov(df_ar.transpose()[2])))



# Defining the density functions of each column of the data. Indeed, density(i, ) is the density function associated to the column i.

def density(i,x):
  prob_density = (1/(np.sqrt(np.pi*2)*sd[i])) * np.exp(-0.5*((x-mean[i])/sd[i])**2)
  return prob_density

In [89]:
# Training a polynomial regression for data.

X = np.array(df[[0,1,2]]).reshape(-1, 3)
Y = np.array(df['y']).reshape(-1, 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)
lin = LinearRegression()
lin.fit(X_train, Y_train)
coeff = np.zeros(4)
coeff[0] = lin.intercept_[0]
coeff[1], coeff[2], coeff[3] = lin.coef_[0][0], lin.coef_[0][1], lin.coef_[0][2]

In [90]:
# Evaluating the model using "r2_score" and "mean_squared_error"

expected_Y  = Y_test
predicted_linear_Y = lin.predict(X_test)
print(metrics.r2_score(expected_Y, predicted_linear_Y))
print(mean_squared_error(expected_Y, predicted_linear_Y))

1.0
2.1367874217626612e-30


In [91]:
from sympy.stats import E, Normal

In [92]:
# Calculating the PEACE's of x0, x1 and x2 on y.

X0, X1, X2 = Normal('X0',mean[0], sd[0]**2), Normal('X1',mean[1], sd[1]**2), Normal('X2',mean[2], sd[2]**2)

def integrand_0(x0,x1,x2):
  return sympy.stats.density(X0)(x0)
def integrand_1(x0,x1,x2):
  return sympy.stats.density(X1)(x1)
def integrand_2(x0,x1,x2):
  return sympy.stats.density(X2)(x2)
print("The PEACE's of x0, x1 and x2 on y, respectively:")
print(E(4*abs(coeff[1])*integrand_0(X0,X1,X2)).evalf(), E(4*abs(coeff[2])*integrand_1(X0,X1,X2)).evalf(), E(4*abs(coeff[3])*integrand_2(X0,X1,X2)).evalf(),'\n', sep='\n')

# Calculating the positive PEACE's of x0, x1 and x2 on y.

print("The positive PEACE's of x0, x1 and x2 on y, respectively:")
print(max(coeff[1],0)*E(4*integrand_0(X0,X1,X2)).evalf(), max(coeff[2],0)*E(4*integrand_1(X0,X1,X2)).evalf(), max(coeff[3],0)*E(4*integrand_2(X0,X1,X2)).evalf(), '\n', sep='\n')

# Calculating the negative PEACE's of x0, x1 and x2 on y.

print("The negative PEACE's of x0, x1 and x2 on y, respectively:")
print(max(-coeff[1],0)*E(4*integrand_0(X0,X1,X2)).evalf(), max(-coeff[2],0)*E(4*integrand_1(X0,X1,X2)).evalf(), max(-coeff[3],0)*E(4*integrand_2(X0,X1,X2)).evalf(),'\n', sep='\n')

The PEACE's of x0, x1 and x2 on y, respectively:
0.496770449484952
1.19318202842525
3.53270260539144


The positive PEACE's of x0, x1 and x2 on y, respectively:
0
1.19318202842525
3.53270260539144


The negative PEACE's of x0, x1 and x2 on y, respectively:
0.496770449484952
0
0




In [93]:
# Calculating the SPACE's of x0, x1 and x2 on y, respectively.

import scipy.optimize
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint

inf = 100
bnds = scipy.optimize.Bounds([-inf, -inf],[inf, inf])

def func_0_real(a):
  return -abs((a[0]-a[1]))*density(0,a[0])*density(0,a[1])
def func_1_real(a):
  return -abs((a[0]-a[1]))*density(1,a[0])*density(1,a[1])
def func_2_real(a):
  return -abs((a[0]-a[1]))*density(2,a[0])*density(2,a[1])

max_0 = -differential_evolution(func_0_real, bnds, seed=1).fun
max_1 = -differential_evolution(func_1_real, bnds, seed=1).fun
max_2 = -differential_evolution(func_2_real, bnds, seed=1).fun

def integrand_max_0(X): 
  return density(1,X[0])*density(2,X[1])

def integrand_max_1(X):
  return density(0,X[0])*density(2,X[1])

def integrand_max_2(X):
  return density(0,X[0])*density(1,X[1])

domain_max = vegas.Integrator([[-inf, inf], [-inf, inf]])
integral_max_0, integral_max_1, integral_max_2 = domain_max(integrand_max_0, nitn=50, neval=10000), domain_max(integrand_max_1, nitn=50, neval=10000), domain_max(integrand_max_2, nitn=50, neval=10000)
print("The SPACE's of x0, x1 and x2 on y, respectively:")
print(4*abs(coeff[1])*max_0*integral_max_0, 4*abs(coeff[2])*max_1*integral_max_1, 4*abs(coeff[3])*max_2*integral_max_2,'\n',sep='\n')

# Calculating the positive SPACE's of x0, x1 and x2 on y, respectively.

print("The positive SPACE's of x0, x1 and x2 on y, respectively:")
print(4*max(coeff[1],0)*max_0*integral_max_0, 4*max(coeff[2],0)*max_1*integral_max_1, 4*max(coeff[3],0)*max_2*integral_max_2,'\n',sep='\n')

# Calculating the negative SPACE's of x0, x1 and x2 on y, respectively.

print("The negative SPACE's of x0, x1 and x2 on y, respectively:")
print(4*max(-coeff[1],0)*max_0*integral_max_0, 4*max(-coeff[2],0)*max_1*integral_max_1, 4*max(-coeff[3],0)*integral_max_2,sep='\n')

The SPACE's of x0, x1 and x2 on y, respectively:
0.360353(45)
0.576613(67)
1.36780(14)


The positive SPACE's of x0, x1 and x2 on y, respectively:
0(0)
0.576613(67)
1.36780(14)


The negative SPACE's of x0, x1 and x2 on y, respectively:
0.360353(45)
0(0)
0(0)
